In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from TextPreprocessing import clean
import numpy as np
from nltk import sent_tokenize
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
traindata=pd.read_json("train-v1.1.json")

In [3]:
def check(sentences):
    new_list=[]
    f=1
    while f==1:
        f=0
        for i in range(len(sentences)-1):
          #print(i)
            if len(sentences[i])==1:
                sentences[i-1]=sentences[i-1]+" " +sentences[i]
                sentences.pop(i)
                break;
            if (sentences[i].endswith('Op.') or sentences[i].endswith('No.') or sentences[i].endswith("!") or sentences[i].endswith(".)") or sentences[i].endswith("no.") or (65<=ord((sentences[i])[-2])<=90)) or sentences[i].endswith("Fr.") or sentences[i].endswith("Capt.")or sentences[i].endswith("Rev.")or sentences[i].endswith("est.")or sentences[i].endswith("app.")or sentences[i].endswith("Lt.")or sentences[i].endswith("Msgr.") or sentences[i].endswith("i.e.") or sentences[i].endswith("Ss.") or sentences[i].endswith("St.") or sentences[i].endswith("Sch.") or sentences[i].endswith("e.g.") or sentences[i].endswith("Mr.") or sentences[i].endswith("Mt.") or sentences[i].endswith("Ecl.") or sentences[i].endswith("Rs.") or sentences[i].endswith("9."):
                sentences[i]=sentences[i]+" " +sentences[i+1]
                sentences.pop(i+1)
                f=1
                break;
            if sentences[i].endswith("?"):
                sentences[i]=sentences[i]+sentences[i+1]
                sentences.pop(i+1)
                break;   
    return sentences

In [4]:
contexts=[]
questions = []
answers_text = []
answers_start = []
for i in range(traindata.shape[0]):
    topic = traindata.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            contexts.append(sub_para['context']) 
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text']) 


In [5]:
no=[5219,8677,8766,8949,20971,32222,34605,34775,36081,36157,36164,36212,36257,38704,39360,40141,41737,42957,44109,45039,45047,45093,47463,48455,48753,48807,53210,53213,53447,54281,55178,55209,55233,55246,55263,55275,55448,55675,56759,57307,58430,59579,62867,63404,65804,68323,68931,69208,69301,70214,70246,70248,70304,70323,70365,70369,70372,70375,70658,70714,70756,72219,72237,72238,72254,72258,72267,72284,72286,72289,72297,73906,75649,77727,79975,80649,81374,81391,81454,82749,85222]
c=0
for i in no:
    contexts.pop(i-c)
    answers_text.pop(i-c)
    questions.pop(i-c)
    c=c+1
print(len(contexts),len(answers_text),len(questions))

87518 87518 87518


In [6]:
df = pd.DataFrame({"context":contexts, "question": questions,"answer_text": answers_text})

In [7]:
df.head(3)

,context,question,answer_text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building


In [40]:
df.to_csv("Squad1.csv",index=False)

In [41]:
paras=list(dict.fromkeys(contexts))
len(paras)

18891

In [42]:
df['sentences'] = df['context'].apply(lambda x: [i for i in check(sent_tokenize(x))])

In [43]:
df.head()

,context,question,answer_text,sentences
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch..."
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch..."
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch..."


In [49]:
sent=list(df["sentences"])

In [51]:
test=" ".join(paras)
sentences=[i for i in check(sent_tokenize(test))]

In [14]:
dataset=sentences+questions
model = Doc2Vec(vector_size=150, min_count=2, epochs=100)
data = [TaggedDocument(doc, [i]) for i, doc in enumerate(dataset)]

In [52]:
len(dataset)

180124

In [16]:
%%time 
model.build_vocab(data)
model.train(data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("Doc2Vec_SQuAD1.model")

Wall time: 33min 28s


In [17]:
sent_vectors=[]
for i in range(len(sentences)):
    sent_vectors.append(model.docvecs[i])

In [18]:
len(sent_vectors)

92606

In [19]:
ques_vectors=[]
for i in range(len(questions)):
    ques_vectors.append(model.docvecs[dataset.index(questions[i])].reshape(1,-1))

In [20]:
len(ques_vectors)

87518

In [21]:
%%time
dict_embeddings_sentences={}
for i in range(len(sentences)):
    #print(i)
    dict_embeddings_sentences[sentences[i]] = sent_vectors[i]

Wall time: 52.9 ms


In [22]:
len(sentences)

92606

In [23]:
len(dict_embeddings_sentences)

92457

In [24]:
import pickle

In [25]:
%%time
dict_embeddings_questions={}
for i in range(len(ques_vectors)):
    #print(i)
    dict_embeddings_questions[questions[i]] = ques_vectors[i]

Wall time: 40.9 ms


In [26]:
with open('doc2vec_sentences.pickle', 'wb') as handle:
    pickle.dump(dict_embeddings_sentences, handle)

In [27]:
with open('doc2vec_questions.pickle', 'wb') as handle:
    pickle.dump(dict_embeddings_questions, handle)